In [1]:
import numpy as np
import json
import pandas as pd
import recordlinkage
import re

read data

In [4]:
with open('/home/p/wines/winerecommender/data-matching/vivino_data_combined_fm.json') as f:
   vivino_data = json.load(f)
vivino_full = pd.read_csv('/home/p/wines/winerecommender/data-matching/vivino_full.csv')
di = pd.read_excel('/home/p/wines/winerecommender/Scraping/Data Sources - scraped/Direkt-Import.xlsx')
dv = pd.read_excel('/home/p/wines/winerecommender/Scraping/Data Sources - scraped/Divino.xlsx')
jacq = pd.read_csv('/home/p/wines/winerecommender/Scraping/jacques.de/jacques.csv', encoding='utf-8')




format vivino data after extraction if necessary

In [5]:
# filter vivino data by keys used for matching, transform to DataFrame and export as .csv
cols = ['wine_id','wine_name','wine_winery', 'wine_country', 'wine_region', 'wine_year', 'wine_type', 'wine_price', 'wine_alcohol']

vivDict_full = [{k: dic[k] for k in vivino_data[0].keys()} for dic in vivino_data]
vivino_df = pd.DataFrame.from_dict(vivDict_full)
vivino_df = vivino_df.set_index('wine_id')
vivino_df.to_csv('/home/p/wines/winerecommender/data-matching/vivino_full.csv')


Define helper functions to format local wine data

In [9]:
#used to match dataframes by relevant columns

def updateWineType(t):
    switch = {
        'Rotwein': 'red',
        'Weißwein': 'white',
        'Weisswein': 'white',
        'Roséwein': 'rosé',
        'red': 'red',
        'white': 'white',
        'rosé': 'rosé'
    }

    try: 
        return switch.get(t.replace(',','').strip().split(' ')[0], 'sparkling')
    except:
        return 


#formats price strings to xx.xx instead of eg. xx,xx$ or $xx.xx. likely unneccessarily complicated but works for now
def formatPrice(x):
    x = str(x).strip()
    if re.match('/^\d+,\d+/', x):
        return float(x.replace('.','').replace(',', '.'))
    else:
        if re.match('/^\d+.\d+$/', x):
            return float(x)
        else:
            if re.match('^\s*$', x):
                return float(0)
            else:
                return float(x.replace('.','').replace(',', '.').replace('€', '').strip())

#removes years from the name, keeps the producer though
def removeYear(x):
    sub = re.sub('\d{4}', '', x)
    return sub.strip()

#formats alcohol percentage

def formatAlcohol(x):
    #if multiple percentages were scraped, just take the first one
    x = str(x).split(', ')[0]
    x = float(x.replace('%', '').replace(',','.').strip())
    return x

#takes year from name string of DirektImport wines

def getYears(names):
    out = []
    for x in names:
        x= str(x)
        m = re.search('\d{4}', x)
        if m:
            out.append(m.group(0))
        else:
            out.append(None)
    return out

#split the string by triple whitespace, then split first element by comma. return first element of that as winery, second element of first split as wine. Works for Direktimport due to consistent naming

def splitWinery(names):
    wineries = []
    wines = [] 

    for x in names:
        w = str(x).strip().split('   ')
        wineries.append(w[0].strip().split(',')[0])
        wines.append(w[1].strip())
    return wineries, wines
#get quality indicators from table
def getQualityIndicators(array):
    seen = set()
    quals_set = []
    for item in array:
        quals = str(item).strip().split(',')
        for qual in quals:
            qual = qual.strip()
            if qual not in seen:
                seen.add(qual)
                quals_set.append(qual)
    return sorted(quals_set)


#remove winery, strip spaces, remove quality sigs
def formatName(x, quals):
    x = removeYear(str(x))
    for qual in quals:
        if re.match(qual, x):
            x = re.sub(qual, '', x)
    re.sub('\s+', ' ', x)
    return x

# replace values in df[col] with values in val if empty/None
def replaceMissingYears(df, col, val):
    indicies_of_missing = df[df[col].isnull()].index 
    for fill_index, dataframe_index in enumerate(indicies_of_missing):
        df.loc[dataframe_index, col] = val[dataframe_index]
    return df


# removes bio certs from jacques wines

def stripBio(x):
    try:
        x = re.sub('\s–\s[\u00C0-\u017Fa-zA-Z\']{2}-[\u00C0-\u017Fa-zA-Z\']{3}-\d{2,3}', '' ,str(x)).strip()
    except Exception as e:
        print(e)
    return x

# removes bottle sizes
def stripSize(x):
    try:
        x = re.sub('\(?\d,\d{1,3}\s*(\S+)\)?', '' ,str(x)).strip()
        x = re.sub('(Magnum)?(flasche)?', '', x).strip()
        x = x.replace('Doppelmagnum', '').strip()
        x = re.sub('\d{1,3}\s*(m)?[lL](iter)?', '', x).strip()
        x = re.sub('.?[gG]oldkapsel.?', '', x).strip()
    except Exception as e:
        print(e)
    return x

#remove weird year entries (e.g. Jahrgangscuvee)
def formatYear(x):
    if re.match('^\d{4}$', str(x).strip()):
        return x
    else:
        return None

# Format Local Wine Data

In [11]:

# ## DO NOT USE AGAIN, extracted regions and formatted alcohol from  early jacques scraper

# def getRegionJacques(series):

#     regions = []
#     for item in series:
#         temp = re.sub(' +', ' ',item.replace("\xa0", "").strip()).split('\n\n')
#         try:
#             regions.append(temp[0].split(', ')[1].strip())
#         except:
#             regions.append('')
#     return regions


jac2 = pd.DataFrame()

#jac2 = pd.read_csv('/home/p/wines/winerecommender/Scraping/jacques.de/jacques.csv')
jac2['wine_country'] = jacq['Herkunft']
jac2['wine_year'] = getYears(jacq['Name'])
jac2['wine_name'] = jacq['Name'].map(lambda x: removeYear(x))
jac2['wine_name'] = jac2['wine_name'].map(lambda x: stripBio(x))
jac2['wine_name'] = jac2['wine_name'].map(lambda x: stripSize(x))
jac2['wine_seller'] = 3
#jac2['wine_region'] = getRegionjacq(jacq['Name'])
jac2['wine_alcohol'] = jacq['Alkoholgehalt'].map(lambda x: formatAlcohol(x))
jac2['wine_price'] = jacq['Preis in euro'].map(lambda x: formatPrice(x))
jac2['wine_type'] = jacq['Typ'].map(lambda x: updateWineType(x))
jac2['wine_description'] = jacq['Beschreibung']
jac2['wine_url'] = jacq['URL']
jac2.applymap(lambda x: x.strip() if isinstance(x, str) else x)

jac2.to_csv('/home/p/wines/winerecommender/data-matching/jacques_formatted.csv')

In [12]:
print(di.columns)
cols=['wine_name','wine_winery', 'wine_country', 'wine_region', 'wine_year', 'wine_type', 'wine_price', 'wine_alcohol', 'wine_description', 'wine_thumb', 'wine_url', 'wine_seller']
di = pd.read_excel('/home/p/wines/winerecommender/Scraping/Data Sources - scraped/Direkt-Import.xlsx')
matching_df = pd.DataFrame(columns=cols)
di['Region:'] = di['Region:'].map(lambda x: str(x).strip().split(', ')[0])
di['Wein:'] = di['Wein:'].map(lambda x: updateWineType(x))
di['Alkohol in %:'] = di['Alkohol in %:'].map(lambda x: formatAlcohol(x))
di['Preis:'] = di['Preis:'].map(lambda x: formatPrice(x))
years_temp = getYears(di['Name:'])
di['Name:'] = di['Name:'].map(lambda x: removeYear(x))
di['Name:'] = di['Name:'].map(lambda x: stripSize(x))

#not sure how to put this in a function
di = replaceMissingYears(di, 'Jahrgang:', years_temp)
di['Jahrgang:'] = di['Jahrgang:'].map(lambda x: formatYear(x))
# this is likely overkill and will produce weird results
# quals = getQualityIndicators(di['Qualität:'])
quals = []

di['Hersteller:'], di['Name:'] = splitWinery(di['Name:'])
di['Name:'] = di['Name:'].map(lambda x: formatName(x, quals))

di_new = di[['Name:', 'Hersteller:', 'Erzeugnis aus:', 'Region:', 'Jahrgang:', 'Wein:', 'Preis:', 'Alkohol in %:', 'Verkostungsnotiz:', 'Image:', 'Details-URL:', 'Seller']]
di_new.columns = cols

#remove sets of wines offered by direktimport
di_new = di_new[~di_new['wine_name'].str.contains('paket')]
di_new = di_new[~di_new['wine_name'].str.contains('Paket')]
di_new.applymap(lambda x: x.strip() if isinstance(x, str) else x)
di_new.to_csv('/home/p/wines/winerecommender/data-matching/direktimport_formatted.csv')
   

Index(['Id', 'Name:', 'Hersteller:', 'Erzeugnis aus:', 'Region:', 'Wein:',
       'Jahrgang:', 'Rebsorte:', 'Qualität:', 'Alkohol in %:', 'Geschmack:',
       'Allergene:', 'Trinktemperatur in °C:', 'Trinkempfehlung:',
       'Dekantieren:', 'Verschluß:', 'Säure g/l:', 'Restzucker g/l:', 'Boden:',
       'Essen:', 'Ausbauart:', 'Auszeichnungen:', 'Paßt zu:', 'Lage:',
       'Herstellung:', 'Produkt aus biologischem Anbau:', 'Biokontrollstelle:',
       'Diverses:', 'Zutaten:', 'Preis:', 'Image:', 'Details-URL:',
       'Verkostungsnotiz:', 'Seller'],
      dtype='object')


In [13]:
print(dv.columns)
dv = pd.read_excel('/home/p/wines/winerecommender/Scraping/Data Sources - scraped/Divino.xlsx')

#missing alcohol, reliable winery info

cols=['wine_name','wine_country', 'wine_region', 'wine_year', 'wine_type', 'wine_price', 'wine_thumb', 'wine_description', 'wine_url', 'wine_seller']

years_temp = getYears(dv['Name'])
dv = replaceMissingYears(dv, 'Jahrgang', years_temp)
dv['Jahrgang'] = dv['Jahrgang'].map(lambda x: formatYear(x))
dv['Art'] = dv['Art'].map(lambda x: updateWineType(x))
dv['Preis'] = dv['Preis'].map(lambda x: formatPrice(x))
dv['Name'] = dv['Name'].map(lambda x: removeYear(x))
dv['Name'] = dv['Name'].map(lambda x: stripSize(x))
dv_new = dv[['Name', 'Herkunftsland', 'Region', 'Jahrgang', 'Art', 'Preis', 'Image', 'Beschreibung', 'Details-URL', 'Seller']]
dv_new.columns = cols

#house brand again
dv_new = dv_new[~dv_new['wine_name'].str.contains('divino.de')]
dv_new.applymap(lambda x: x.strip() if isinstance(x, str) else x)
dv_new.to_csv('/home/p/wines/winerecommender/data-matching/divino_formatted.csv')



Index(['Id', 'Name', 'Jahrgang', 'Rebsorte', 'Weingut', 'Produktbezeichnung',
       'Ort', 'Passt gut zu', 'Herkunftsland', 'Anlass', 'Größe des Winzers',
       'Winzer', 'Region', 'Herstellungsmethode', 'Art', 'Lagerfähigkeit',
       'Charakteristik', 'Preis', 'Image', 'Details-URL', 'Beschreibung',
       'Seller'],
      dtype='object')


combine local wines to singular DF/table for DB dump

In [21]:
local_cols = ['wine_name','wine_country', 'wine_region', 'wine_year', 'wine_type', 'wine_price', 'wine_url', 'wine_thumb', 'wine_description', 'wine_seller']

direktimport = pd.read_csv('/home/p/wines/winerecommender/data-matching/direktimport_formatted.csv', encoding='utf-8')
divino = pd.read_csv('/home/p/wines/winerecommender/data-matching/divino_formatted.csv', encoding='utf-8')
jacques = pd.read_csv('/home/p/wines/winerecommender/data-matching/jacques_formatted.csv', encoding='utf-8')

direktimport["wine_name"] = direktimport["wine_winery"] + ' ' + direktimport["wine_name"]
jacques["wine_region"] = None
jacques["wine_thumb"] = None
divino = divino[local_cols]
direktimport = direktimport[local_cols]
jacques = jacques[local_cols]

local_df = pd.DataFrame(columns= local_cols)
local_df = local_df.append(direktimport)
local_df = local_df.append(divino)
local_df = local_df.append(jacques)
local_df["taste_id"] = None
local_df = local_df.dropna(subset=['wine_country'])
local_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
print(local_df.shape)

local_df.to_csv('/home/p/wines/winerecommender/data-matching/local_wines_formatted.csv')


(1326, 11)


# match local data
We scraped data from Vivino in multiple iterations due to the present complexity, aiming to achieve sufficient coverage for local wines.
These cells were used for the first iteration, in order to determine the amount of required re-scraping to be done based on each vendor's present inventory.
Where possible, we matched winery and wine name seperately to increase accuracy, though this was only an option for Wein-Direktimport as both Jacques' and Divino's scraping data proved nigh impossible to work with in this regard. 
For these cases, the strings were combined to a full wine name.
Additionally, some manual data cleaning was performed on the smaller Divino & Jacques sets to eliminate size denotations (e.g. Magnum, 0,5 L, 1 Liter). It would have been possible to automate this using RegEx, but the amount of effort required for all edge cases is not in proportion to the requirements of this project.

In [29]:
import numpy as np
import json
import pandas as pd
import recordlinkage
import re
import datasketch
import fuzzymatcher

vivino = pd.read_csv('/home/p/wines/winerecommender/data-matching/vivino_full.csv', encoding='utf-8')

direktimport = pd.read_csv('/home/p/wines/winerecommender/data-matching/direktimport_formatted.csv', encoding='utf-8')
divino = pd.read_csv('/home/p/wines/winerecommender/data-matching/divino_formatted.csv', encoding='utf-8')
jacques = pd.read_csv('/home/p/wines/winerecommender/data-matching/jacques_wines.csv', encoding='utf-8')

match vivino and direktimport

In [30]:
ons = ['wine_name', 'wine_winery', 'wine_country', 'wine_region', 'wine_type']
vivinof_di = vivino[ons]
direktimportf = direktimport[ons]

resf_di = fuzzymatcher.fuzzy_left_join(direktimportf, vivinof_di, ons, ons)
resf_di.sort_values('best_match_score', ascending = False, inplace=True)
resf_di.to_csv('/home/p/wines/winerecommender/data-matching/fuzzymatch_results/fuzzy_results_di.csv')

KeyboardInterrupt: 

this resulted in about 170 usable matches, the rest needed to be rescraped. 

In [13]:
X = 170
resf_di['search'] = resf_di['wine_winery_left'] +' '+ resf_di['wine_name_left']
resf_di['index'] = list(range(len(resf_di['search'])))
resf_di.set_index('index', inplace=True)
strings = list(resf_di['search'][X:])
ids = list(resf_di['__id_left'].map(lambda x: x.split('_')[0])[X:])
i=0
out = []
for string in strings:
    out.append({
        "local_id": ids[i],
        "search_string": string
    })
    i+=1

with open("/home/p/wines/winerecommender/data-matching/rescrape/search_di.json", "w") as f:
    f.write(json.dumps(out))

match vivino and divino

In [15]:
vivino["full_name"] = vivino["wine_winery"] + ' ' + vivino["wine_name"]
ons = ['full_name', 'wine_country', 'wine_region', 'wine_type']
vivinof_dv = vivino[ons]
divino["full_name"] = divino["wine_name"]
divinof = divino[ons]
resf_dv = fuzzymatcher.fuzzy_left_join(divinof, vivinof_dv, ons, ons)
resf_dv.sort_values('best_match_score', ascending = False, inplace=True)
resf_dv.to_csv('/home/p/wines/winerecommender/data-matching/fuzzymatch_results/fuzzy_results_dv.csv')


this resulted in approx. 9 usable matches, with the rest needing rescraping

In [18]:
x = 9
resf_dv['search'] = resf_dv['full_name_left']
resf_dv['index'] = list(range(len(resf_dv['search'])))
resf_dv.set_index('index', inplace=True)
strings = list(resf_dv['search'][x:])
ids = list(resf_dv['__id_left'].map(lambda x: x.split('_')[0])[x:])
i=0
out = []
for string in strings:
    out.append({
        "local_id": ids[i],
        "search_string": string
    })
    i+=1

with open("/home/p/wines/winerecommender/data-matching/rescrape/search_dv.json", "w") as f:
    f.write(json.dumps(out))

match jacques and vivino

In [25]:
vivino["full_name"] = vivino["wine_winery"] + ' ' + vivino["wine_name"]
ons = ['full_name', 'wine_country', 'wine_type']
jacques["full_name"] = jacques["wine_name"]
vivinof_j = vivino[ons]
jacquef = jacques[ons]
resf_j = fuzzymatcher.fuzzy_left_join(jacquef, vivinof_j, ons, ons)
resf_j.sort_values('best_match_score', ascending = False, inplace=True)
resf_j.to_csv('/home/p/wines/winerecommender/data-matching/fuzzymatch_results/fuzzy_results_j.csv')


this resulted in 25 usable matches, with the rest needing rescraping

In [26]:
x = 25
resf_j['search'] = resf_j['full_name_left']
resf_j['index'] = list(range(len(resf_j['search'])))
resf_j.set_index('index', inplace=True)
strings = list(resf_j['search'][x:])
ids = list(resf_j['__id_left'].map(lambda x: x.split('_')[0])[x:])
i=0
out = []
for string in strings:
    out.append({
        "local_id": ids[i],
        "search_string": string
    })
    i+=1

with open("/home/p/wines/winerecommender/data-matching/rescrape/search_j.json", "w") as f:
    f.write(json.dumps(out))

# match full table after rescraping
After scraping more wines from vivino, we look to match as many wines as possible between out local set and our vivino set.

In [37]:
import fuzzymatcher
local = pd.read_csv('/home/p/wines/winerecommender/data-matching/local_wines_formatted.csv')
vivino = pd.read_csv('/home/p/wines/winerecommender/data-matching/vivino_full.csv', encoding='utf-8')
vivino["wine_name"] = vivino["wine_winery"] + ' ' + vivino["wine_name"]
vivino.set_index("wine_id")
ons = ['wine_name', 'wine_country', 'wine_type']

resf = fuzzymatcher.fuzzy_left_join(local, vivino, ons, ons)
local["taste_id"] = list(resf["wine_id"])
resf.sort_values('best_match_score', ascending = False, inplace=True)
resf.to_csv('/home/p/wines/winerecommender/data-matching/fuzzymatch_results/fuzzy_results_all.csv')
local.to_csv('/home/p/wines/winerecommender/data-matching/fuzzymatch_results/local_wines_matched.csv')



